In [1]:
from ipywidgets import Layout, Button, VBox, Label, HBox
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from ipywidgets import Button, HBox, VBox, Layout , FileUpload
import numpy as np
import xml.etree.ElementTree as ET
from skimage import io, feature, morphology
from skimage import measure
from  scipy import ndimage as nd
from skimage.color import label2rgb
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage.measure import label, regionprops, points_in_poly
from skimage import data, filters, measure, morphology
from PIL import Image
import base64
from io import BytesIO

layout = Layout(overflow_x='hidden',
                    overflow_y='auto',
                    border='3px solid black',
                    width='152px',
                    height='35px',
                    flex_flow='column',
                    display='flex')

layoutfull = Layout(overflow_x='hidden',
                    overflow_y='auto',
                    border='3px solid black',
                    width='550px',
                    height='35px',
                    flex_flow='column',
                    display='flex')

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='100%')

app_layout = Layout(border='3px solid black',
                    width='550px'
                   )
gen_layout = Layout(width='550px'
                   )

path = "./DB_12215/"




In [2]:
# Create global material table
materiauTable = pd.DataFrame(columns = ["Produit","Matiere","Reference","Masse volumique ρ","Épaisseur du stratifié par masse de fibre sèche t/w","Module d’élasticité E Ef1//Fibres","Module d’élasticité E Ef2⏊ Fibres","Module de cisaillement G","Coefficient de Poisson ν","contrainte de cisaillement interlaminaire τUIL","εut1","εut2","εuc1","εuc2","εuf1b","εuf2c","γu12","σut1","σut2","σuc1","σuc2","σuf1","σuf2","τ12"])

#read ISO11215-C.4
file = "table C.4.csv"
table = pd.read_csv(path + file)

#read ISO11215-C.6
file = "table C.6.csv"
table6 = pd.read_csv(path + file)



# filter renfort & matrices
liste_renfort = table[(table['Type'] == 'Fibres de renfort')]
liste_matrix = table[(table['Type'] == 'Matrices')]

In [3]:


# Built tip list
tip_renfort = "Masse volumique ρ (ρf ou ρm): " \
    +liste_renfort['Masse volumique ρ (ρf ou ρm)'] \
    + table.iloc[0,2] \
    + '\n' \
    +"Module d’élasticité E Ef1//Fibres: " \
    +liste_renfort['Module d’élasticité E Ef1//Fibres'] \
    + table.iloc[0,3] \
    + '\n-\n' \
    +liste_renfort['Reference'] 
tip_renfort = tip_renfort.tolist()

# Create renfort button
renfortButton = widgets.ToggleButtons(
    options=liste_renfort['Denomination'],
    description='Renforts:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=tip_renfort,
    layout=layout,
#     icons=['check'] * 3
)


In [4]:
# Create matrix button
tip_matrix = "Masse volumique ρ (ρf ou ρm): " \
    +liste_matrix['Masse volumique ρ (ρf ou ρm)'] \
    + table.iloc[0,2] \
    + '\n' \
    +"Module d’élasticité E Ef1//Fibres: " \
    +liste_matrix['Module d’élasticité E Ef1//Fibres'] \
    + table.iloc[0,3] \
    + '\n-\n' \
    +liste_matrix['Reference'] 
tip_matrix = tip_matrix.tolist()
matrixButton = widgets.ToggleButtons(
    options=liste_matrix['Denomination'],
    description='Matrices:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=tip_matrix,
    layout=layout,
#     icons=['check'] * 3
)

In [5]:
# Create phi button
phiButton = widgets.FloatSlider(
    value=7.5,
    min=0.05,
    max=0.95,
    step=0.001,
    description='Taux de fibre',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [6]:
# create one pd dataframe with one column
options = pd.DataFrame(columns = ['Produit'])

# Feed data frame
options['Produit'] = ['Unidirectionnel\r«UD»' \
            ,'Mat de verre\r «CSM»\r' \
            ,'Roving tissé' \
            ,'Bidirectionnel 0/90°\r «BD+»' \
            ,'Double biais ±45\r «DBx»' \
            ,'Quadriaxial\r0/45/90/_45\r «Qx»']


# Create produit button
produitButton = widgets.ToggleButtons(
    options=options['Produit'],
    description='Produit:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['1','2','3','4','5','6','7'],
    layout=layout,
#     icons=['check'] * 3
)


In [7]:
# Keep created material
keepButton = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Conserve le materiau cree',
    icon='check'
)

In [8]:
# Calcul des proprietes mecaniques des produit
def calc_mech_prop(renfort,matrix,produit,compound,value):
    
    # Formatage des entrees
    Ef1 = np.float(renfort[1]) 
    Ef2 = np.float(renfort[2]) 
    Em = np.float(matrix[1])
    Gf = np.float(renfort[3])
    Gm = np.float(matrix[3])
    nuf = np.float(renfort[4])
    num = np.float(matrix[4])
    rhof = np.float(renfort[0])
    rhom = np.float(matrix[0])
    epsilonUt1 = np.float(compound.iloc[0,2])
    epsilonUt2 = np.float(compound.iloc[0,3])
    epsilonUc1 = np.float(compound.iloc[0,4])
    epsilonUc2 = np.float(compound.iloc[0,5])
    epsilonUf1 = np.float(compound.iloc[0,6])
    epsilonUf2 = np.float(compound.iloc[0,7])
    tauU12 = np.float(compound.iloc[0,8])
    
    
    #Calcul de tsurw selon le choix unite + forcage tvf
    unite = 'tvf'
    if unite == 'tmf':
        phi = value/(value+(1-value)*rhof/rhom)
    else:
        phi = value
    tsurw = 1/phi/rhof
    rhoc = phi*rhof + (1-phi)*rhom
    
    #Calcul de la valeur maximum de cisaillement interlaminaire
    tauUIL = 22.5-33*phi/(phi+0.89)
    
    EUD1 = 0.975*(Ef1*phi+Em*(1-phi))
    etaE = (Ef2/Em-1)/(Ef2/Em+1)
    EUD2 = Em*(1+etaE*phi)/(1-etaE*phi)
    etaG = (Gf/Gm-1)/(Gf/Gm+1)
    GUD = Gm*(1+etaG*phi)/(1-etaG*phi)
    nuUD12 = nuf*phi+num*(1-phi)
    nuUD21 = nuUD12*EUD2/EUD1
    #return EUD1,EUD2,GUD,nuUD12,nuUD21,tauUIL

    #calc_mech_prop_CSM(EUD1,EUD2):
    ECSM = 3/8*EUD1+5/8*EUD2
    GCSM = 1/8*EUD1-1/4*EUD2
    # QuestionGregoire nuCSM tableauC.5 formuleisOK ?
    nuCSM = ECSM/2/(GCSM-1)
    #return ECSM, GCSM, nuCSM

    # QuestionGregoire GUD12 tableauC.5 correcttypois GUD12 or GUD ?
    #calc_mech_prop_BDP(EUD1,EUD2,GUD):
    EBDP = 0.5*(EUD1 + EUD2)
    GBDP = GUD
    nuBDP = nuUD12*EUD2/EBDP
    #return EBDP,GBDP,nuBDP

    #calc_mech_prop_DBX(EBDP,GBDP,nuBDP):
    EDBX = 4*EBDP/(EBDP/GBDP+2*(1-nuBDP))
    GDBX = EBDP/(1+nuBDP)/2
    nuDBX = EDBX/4/EBDP*(EBDP/GBDP-2*(1-nuBDP))
    #return EDBX, GDBX, nuDBX

    #calc_mech_prop_QDX(EBDP,EDBX,GBDP,GDBX,nuBDP,nuDBX):
    A11 = EBDP/(1-nuBDP**2)+EDBX/(1-nuDBX**2)
    A12 = nuBDP*EBDP/(1-nuBDP**2)+nuDBX*EDBX/(1-nuDBX**2)
    EQDX = 0.5*(A11-A12**2/A11)
    GQDX = 0.5*(GBDP + GDBX)
    nuQDX = A12/A11
    #return EQDX,GQDX,nuQDX
    
    if produit == 'Unidirectionnel\r«UD»':
        
        E1 = EUD1
        E2 = EUD2
        G12 = GUD
        nu12 = nuUD12
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Mat de verre\r «CSM»\r':
        
        E1 = ECSM
        E2 = ECSM
        G12 = GCSM
        nu12 = nuCSM
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Roving tissé':
        
        E1 = EBDP
        E2 = EBDP
        G12 = GBDP
        nu12 = nuBDP
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
    
    elif produit == 'Bidirectionnel 0/90°\r «BD+»':
        
        E1 = EBDP
        E2 = EBDP
        G12 = GBDP
        nu12 = nuBDP
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Double biais ±45\r «DBx»':
        
        E1 = EDBX
        E2 = EDBX
        G12 = GDBX
        nu12 = nuDBX
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Quadriaxial\r0/45/90/_45\r «Qx»':
        
        E1 = EQDX
        E2 = EQDX
        G12 = GQDX
        nu12 = nuQDX  
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
        
    

In [9]:
def on_value_change(change):
    global table
    global table6
    global materiauTable
    # extraction caracteristiques renfort
    renfortchoix = renfortButton.value
    extract = table[(table['Denomination'] == renfortchoix)]
    renfort = extract.iloc[0,3:8].tolist()
    # extraction caracteristiques matrices
    matrixchoix = matrixButton.value
    extract = table[(table['Denomination'] == matrixchoix)]
    matrix = extract.iloc[0,3:8].tolist()
    # already a float
    produit = produitButton.value
    # already a float
    value = phiButton.value
    # extraction des valeurs de deformations ultimes
    compound = table6[(table6['Produit'] == produit)&(table6['Matiere'] == renfortchoix + ' et ' + matrixchoix)]
    #epsilonUt1 = np.float(compound.iloc[0,2])

    #Execute kernel
    res = calc_mech_prop(renfort,matrix,produit,compound,value)

    #Dynamic plot update
    with a.batch_update():
        a.data[0].x=[np.float(res[1])]
        b.data[0].x=[np.float(res[2])]
        c.data[0].x=[np.float(res[9])]
        d.data[0].x=[np.float(res[13])]
        
renfortButton.observe(on_value_change, names='value')
matrixButton.observe(on_value_change, names='value')
produitButton.observe(on_value_change, names='value')
phiButton.observe(on_value_change, names='value')


In [10]:
#Boat parameters importation
#tree = ET.parse('mach40.iscn')
#root = tree.getroot()
#placeholder  = [widgets.HTML(root[i].tag) for i in [9,12]]
#value = [widgets.Text(root[i].text) for i in [9,12]]
#left = widgets.VBox(placeholder)
#right = widgets.VBox(value)
#boatParameterMenu1 = widgets.HBox([left,right],layout={'width': 'max-content'})



# Create parameter slider
lwlButton = widgets.FloatSlider(
    value=2.500,
    min=2.5,
    max=24.0,
    step=0.001,
    description='Lwl:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

# Create parameter slider
mldcButton = widgets.FloatSlider(
    value=10000,
    min=10,
    max=100000,
    step=1,
    description='mLDC:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

# Create parameter slider
speedButton = widgets.FloatSlider(
    value=1,
    min=0,
    max=100,
    step=1,
    description='V:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)


In [11]:
#Boat Characteristics input section components

boatcharacteristics1 = widgets.HTML("<font size=7>12215_alive!</font><font size=3>  by A.C.E. CATTELLIOT ENGINEERING</font><br>" +
                                         "<br>"+"<br>"+"<br>"+
                                         "<font size=5>DESCRIBE YOUR BOAT</font><br>"+
                                         "<font size=3>Among all the principal datas that drive the 12215, the waterline lenght and the maximum displacement in fully loaded condition are the most influent. Be assisted by our interactive user entry that will give you feedback of your values compare to more than a thousand existing boats already fully EC-certified as productions or custom boats."+
                                         "<br>",layout=gen_layout
                                        )
boatcharacteristics2 = widgets.HTML("<font size=3><br>Beyong these bounds, if your boat project is in the scope of the standard, as given in <font size=2>ISO12215-5 1 Scope</font>, the standard should apply taking also into considerations extra analysis that should be done by other described methods.</font>",layout=gen_layout)
boatcharacteristics3 = widgets.HTML("<font size=3><br>Displacement of the boat in kg</font>,<br><font size=2> including all appendages, in fully loaded condition, should be given according to <font size=2>ISO 8666 Small craft — Principal data</font><font size=2>. The maximum and minimum values of the displacement slider are automatically adjusted based on the displacement in D category of 925 sailing boats and 276 motor boats including semi-rigids.</font>",layout=gen_layout )
boatcharacteristics4 = widgets.HTML("<font size=3><br>Lenght of the waterline in meters</font>,<br><font size=2> boat upright in fully loaded condition should be given according to <font size=2>ISO12215-5 3 Terms and definitions -Table 1.</font>",layout=gen_layout )

boatcharacteristics5 = widgets.HTML("<font size=1>Scroll and click to valid your choice</font>"
                                        )

boatcharacteristics6 = widgets.HTML("<font size=1>User the slider or enter directly the value on the right to enter the value</font>"
                                        )
boatcharacteristics7 = widgets.HTML("<font size=3><br>Design Category</font>,<br><font size=2> should be given according to the European Recreational Craft Directive 2013/53/EU <font size=2>Annex 1 - Essential requirements.</font>",layout=gen_layout )

boatcharacteristics8 = widgets.HTML("<font size=3><br>Boat type</font>,<br><font size=2> Choose from pre-segmented categories of products that reflects the market as analysed in 2020 by the Federation des Industries Nautiques in his annual report.</font>",layout=gen_layout )
boatcharacteristics9 = widgets.HTML("<font size=3><br>Boat speed</font>,<br><font size=2> Maximum speed in fully loaded conditions, in kts should be given according to <font size=2>ISO12215-5 3 Terms and definitions -Table 1.</font>",layout=gen_layout )

boatcharacteristics10 = widgets.HTML("Nothing to display yet.",layout=app_layout)
boatcharacteristics11 = widgets.HTML("Nothing to display yet.",layout=app_layout)

In [12]:
#Building

buildingcharacteristics1 = widgets.HTML("<br>"+"<br>"+"<br>"
                                         "<font size=5>EXPLAIN HOW IT IS BUILD</font><br>"+
                                         "<font size=3>The materials available are taken according to  <font size=1>ISO12215-5 Table 17</font>. You can also use other materials, including new fibres and resins, provided that they show similar cohesion, durability, resistance to marine environment and elongation at break as the ones quoted in the standard."+
                                         "<br>",layout=gen_layout
                                        )

buildingcharacteristics2 = widgets.HTML("<br><font size=3>Building material</font>,<font size=2> should be given according to <font size=1>ISO12215-5 5.1 Materials.</font>",layout=gen_layout )

buildingcharacteristics3 = widgets.HTML("<br><font size=3>Manufacturing process</font>,<font size=2> should be given according to <font size=1>ISO12215-5 Table 14.</font>. The usage of the parameter has been extended to metallic and wooden constructions.",layout=gen_layout )

buildingcharacteristics4 = widgets.HTML("<br><font size=3>Material quality</font>,<font size=2> should be given according to <font size=1>ISO12215-5 5.1 Table 15.</font> The usage of the parameter has been extended to metallic and wooden constructions.",layout=gen_layout )

buildingcharacteristics5 = widgets.HTML("Nothing to display yet.",layout=app_layout)

#read csv
file = "listeMateriau.csv"
listeTypeBateau = pd.read_csv(path + file)


# Create listeTypeBateau button
listeMateriau = widgets.ToggleButtons(
    options=listeTypeBateau['Matériau'],
    description='listeMateriau:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "550px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

#read csv
file = "listeProcede_de_stratification.csv"
listeProcede = pd.read_csv(path + file)


# Create listeTypeBateau button
listeProcedeButton = widgets.ToggleButtons(
    options=listeProcede['Procédé de fabrication'],
    description='listeProcess:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "550px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

#read csv
file = "listeQualite_du_materiau.csv"
listeQualite = pd.read_csv(path + file)


# Create listeTypeBateau button
listeQualiteButton = widgets.ToggleButtons(
    options=listeQualite['Qualité du matériau'],
    description='listeQualite:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "550px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

#read csv
file = "Tableau 14.csv"
listeBuildingQualite = pd.read_csv(path + file)

def update_qualite(self):
    global listeBuildingQualite
    if (listeQualiteButton.value != 'Mesurée')&(listeProcedeButton.value != "Chaudronnerie, soudure"):
        aaa = listeBuildingQualite[(listeBuildingQualite['Qualité du matériau']==listeQualiteButton.value)&(listeBuildingQualite['Procédé de fabrication']==listeProcedeButton.value)]
    else:
        aaa = listeBuildingQualite[(listeBuildingQualite['Qualité du matériau']== 'Mesurée')&(listeBuildingQualite['Procédé de fabrication']==listeProcedeButton.value)]
        aaa['Mesures mises en œuvre par le constructeur'].values[0] = "Les propriétés mécaniques du produits, à l’état construit sont déterminés sur la base d’essai des caractéristiques mécaniques (module, allongement à la rupture, etc.)et d'enregistrements de suivi, de traçabilité des lots de matériaux. Les essais sont réalisés à la fréquence d'au moins une fois par an). Les procédures d’essai doivent figurer dans un manuel Qualité, conformément à <font size=1>ISO 9001 ou équivalent</font>."
    
    buildingcharacteristics5.value = "<font size=2>The person or company that endorse the responsability of the boat construction should demonstrate that the following quality requirements are actually has properly followed the state of the art in terms of building technics,  <br>—environment (e.g. temperature, hygrometry, etc. during storage and building) <br>— building process (e.g. preparation process prior to building such as dusting, degreasing, priming, etc.)<br>— type of material (e.g. proper combination of material, other relevant) :<br> "+aaa['Mesures mises en œuvre par le constructeur'].values[0]+".</font>"
    
listeQualiteButton.observe(update_qualite)
listeProcedeButton.observe(update_qualite)

In [13]:
#read csv
file = "listeTypeBateau.csv"
listeTypeBateau = pd.read_csv(path + file)


# Create listeTypeBateau button
listeTypeBateauButton = widgets.ToggleButtons(
    options=listeTypeBateau['Type'],
    description='listeTypeBateau:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "550px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

# create one pd dataframe with one column
listeDesignCategory = pd.DataFrame(columns = ['Catégorie de navigation'])


# Feed data frame

#read csv
file = "listeDesignCategory.csv"
listeDesignCategory = pd.read_csv(path + file,delimiter=",")


# Create design category button
listeDesignCategoryButton = widgets.ToggleButtons(
    options=listeDesignCategory['Design category'],
    description='Design category:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "50px"},
    #tooltips='',
    layout=layoutfull,
#      icons=['check'] * 3
)               
               


In [14]:
def update_minmax(self):
    global listeTypeBateau
    lwlButton.min = 0.0
    lwlButton.max = 25.00
    minnew = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,1]
    maxnew = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,2]
    lwlButton.min = minnew
    lwlButton.max = maxnew 
    return 

listeTypeBateauButton.observe(update_minmax)
    
    

In [15]:
# upload image management

# create callback function for uploader            
def response(change):
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    with open('myfile.png', 'wb') as f: f.write(content)
    text = io.imread('myfile.png')
    
    pil_img = Image.fromarray(np.flipud(text)) # PIL image object
    prefix = "data:image/png;base64,"
    with BytesIO() as stream:
        pil_img.save(stream, format="png")
        base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")
    with lol.batch_update():
        lol.data = []
        #details0.value = uploaded_filename+" has been uploaded."
        lol.add_trace(go.Image(source = base64_string))
        
uploader = FileUpload(multiple=False,description='Upload image')
uploader.observe(response, names='value')
        
# create root figure
layout6 = go.Layout(title='',
                    width=550,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=5,
                                pad=4
                                ),
                   )



lol = go.FigureWidget(layout=layout6)

# define root figure layout
lol.layout.hovermode = 'closest'
lol.update_yaxes(scaleanchor = "x", scaleratio = 1)
lol.layout.showlegend = False
lol.layout.plot_bgcolor = 'white'
#lol.layout.width=550



In [16]:
# create design category table
trace5 = go.Table(
    header=dict(values=['Wind force in Beaufort','Significant wave height H ⅓ in metres'],
                #fill_color='paleturquoise',
                #align=['left','left','left','center','center','center'],
               ),
    cells=dict(#values=materiauTable.transpose().values.tolist(),
                values = ['',''],
                #fill_color='lavender',
                #align=['left','left','left','center','center','center'],
                #format = [None,None,None,".3f",".3f",".0f"]),
                ))
layout5 = go.Layout(title='',
                    width=550,
                    height=65,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=5,
                                pad=4
                                ),
                   )
categorytable = go.FigureWidget(data=[trace5], layout=layout5)


def update_category(self):
    global listeDesignCategory
    aaa = listeDesignCategory[listeDesignCategory['Design category']==listeDesignCategoryButton.value]
    categorytable.update_traces(cells_values=aaa[['Wind force in Beaufort','Significant wave height H ⅓ in metres']].transpose().values.tolist(), selector=dict(type='table'))
    boatcharacteristics10.value = "<font size=2>"+aaa['notes'].values[0]+" The boat should be designed and constructed to withstand the wind speed and sea state ranges given below while maintaining the manoeuvrability.</font>"
    
listeDesignCategoryButton.observe(update_category)


In [17]:
# fonction d'analyse bateau
def navprog(mldc,lwl):
    res = mldc/lwl**2.4
    if res < 12.5:
        prog='Exreme'
    elif (res >=12.5)&(res <14):
        prog='Racing'
    elif (res >=14.0)&(res <27.0):
        prog='Gran Turismo'
    elif (res >=27.0):
        prog='Grand cruiser'
    return res,prog
    
def navmode(V,lwl):
    res = V/lwl**.5
    if res >= 5:
        mode='Planning'
    else:
        mode='Displacement'
    return res,mode

def mldc2as(mldc):
    res = 0.07*mldc**(2/3)
    return res

In [18]:
def lwl2mldc(lwl,a,b,c,d):
    res = d + (a - d)/(1 + (lwl/b)**c)
    return res

def update_mldc(self):
    global listeTypeBateau
    mldcButton.min = 0.0
    mldcButton.max = 100000
    au = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,3]
    bu = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,4]
    cu = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,5]
    du = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,6]
    al = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,7]
    bl = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,8]
    cl = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,9]
    dl = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,10]
    #trick to correct functionnality for very small unit.
    mldcButton.min = min(lwl2mldc(lwlButton.value,al,bl,cl,dl),lwl2mldc(lwlButton.value,au,bu,cu,du))
    mldcButton.max = max(lwl2mldc(lwlButton.value,al,bl,cl,dl),lwl2mldc(lwlButton.value,au,bu,cu,du))
    

def analyse_description(self):
    global listeTypeBateau
    propulsion = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,11]
    res1 = navprog(float(mldcButton.value),float(lwlButton.value))
    res2 = navmode(float(speedButton.value),float(lwlButton.value))
    As = mldc2as(float(mldcButton.value))
    Astext = str('{:.1f}'.format(As))+'m2'
    propulsiontext0 = "The principal means of propulsion is :"+propulsion
    progtext0 = "The navigation programm : "+res1[1]
    modetext0 = "The mode of navigation is :"+res2[1]+" according to "
    
    if propulsion == 'Sailing':
               propulsiontext1 = ". The total upwind sail area of the boat should be greater than "+Astext+". If it is not the case, your boat should be considered as a motorboat, according to <font size=2>ISO12215-5 3 Terms and definitions</font>"
    
    elif propulsion == 'Motor':
             propulsiontext1=". However, if your boat is equipped with a sailing rig, the total surface of the boat should be lesser than "+Astext+". If it is not the case, your boat should be considered as a sailing boat, according to <font size=2>ISO12215-5 3 Terms and definitions</font>"
    
    if res2[1] == 'Displacement':
              modetext1 = "<font size=2>ISO12215-5 3 Terms and definitions</font>, its mass should be mainly supported by buoyancy forces."
    
    elif res2[1] == 'Planning':
              modetext1 = "<font size=2>ISO12215-5 3 Terms and definitions</font>, its mass should be significantly supported by forces coming from dynamic lift due to speed in the water."
    
    
    
    boatcharacteristics11.value = "<br><font size=2>" + progtext0 +"</font>." +"<br>" \
                                     +"<font size=2>" + propulsiontext0 + propulsiontext1 +"</font>" +"<br>" \
                                     +"<font size=2>" + modetext0 + modetext1 +"</font>" +"<br>"
    return
    
    
lwlButton.observe(update_mldc)
mldcButton.observe(analyse_description)
speedButton.observe(analyse_description)
lwlButton.observe(analyse_description)

In [19]:
# Make the app

# preparation
lol.update_xaxes(showticklabels=False)
lol.update_yaxes(showticklabels=False)

trace1 = go.Bar( x = [0.01],y = [""], orientation='h')
layout1 = go.Layout(title='Épaisseur du stratifié par Masse de fibre sèche t/w',
                   xaxis=dict(range=[0,3]),
                   xaxis_title="longueur en millimetres",
                   width = 550,
                   height = 200,
                   title_x = 0,
                   legend_title_font_size=8)
a = go.FigureWidget(data=[trace1],layout=layout1)
trace2 = go.Bar( x = [0.01],y = [""], orientation='h')
layout2 = go.Layout(title="Module d’élasticité E Ef1//Fibres",
                   xaxis=dict(range=[0,1e5]),
                   xaxis_title="module en MPa",
                   width = 550,
                   height = 200,
                   title_x = 0,
                   legend_title_font_size=8)
b = go.FigureWidget(data=[trace2],layout=layout2)
trace3 = go.Bar( x = [0.01],y = [""], orientation='h')
layout3 = go.Layout(title="σuc1",
                   xaxis=dict(range=[0,1e5]),
                   xaxis_title="contrainte en MPa",
                   width = 550,
                   height = 200,
                   title_x = 0,
                   legend_title_font_size=8)
c = go.FigureWidget(data=[trace3],layout=layout3)
trace4 = go.Bar( x = [0.01],y = [""], orientation='h')
layout4 = go.Layout(title="τ12",
                   xaxis=dict(range=[0,1e5]),
                   xaxis_title="contrainte en MPa",
                   width = 550,
                   #height = 800,
                   title_x = 0,
                   legend_title_font_size=8)
d = go.FigureWidget(data=[trace4],layout=layout4)

#k = HBox([listeTypeBateauButton,listeDesignCategoryButton],layout={'width': 'max-content'})
i1 = HBox(children=[a,b])
i2 = HBox(children=[c,d])
i = VBox([i1,i2])
h = HBox([produitButton,renfortButton,matrixButton])
g = HBox([phiButton,keepButton],layout=box_layout)
VBox(children=[boatcharacteristics1,
               uploader,
               lol,
               boatcharacteristics7,
               boatcharacteristics5,
               listeDesignCategoryButton,
               boatcharacteristics10,
               categorytable,
               boatcharacteristics8,
               boatcharacteristics5,
               listeTypeBateauButton,
               boatcharacteristics4,
               boatcharacteristics6,
               lwlButton, 
               boatcharacteristics3,
               boatcharacteristics6,
               mldcButton,
               boatcharacteristics9,
               boatcharacteristics6,
               speedButton,
               boatcharacteristics11,
               #boatcharacteristics2,
               buildingcharacteristics1,
               buildingcharacteristics2,
               boatcharacteristics5,
               listeMateriau,
               buildingcharacteristics3,
               boatcharacteristics5,
               listeProcedeButton,
               buildingcharacteristics4,
               boatcharacteristics5,
               listeQualiteButton,
               buildingcharacteristics5,
               #i,
               #g,
               #h],
              ],
                layout=Layout(
                                display='flex',
                                flex_flow='column',
                                #border='solid 2px',
                                justify_items='flex-start',
                                align_items='stretch',
                                width='75%'
                           )
     )

